#기본 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/ssac/deep/"

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import tensorflow as tf
import numpy as np
import random
import torch
import os

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
tf.random.set_seed(seed)

In [ ]:
# 이미지의 가로 세로
IMG_SIZE = 224
# 비디오에서 학습할 프레임 개수
MAX_SEQ_LENGTH = 20

In [ ]:
train_df = pd.read_csv(data_path + 'train.csv')
train_df["label"] = train_df["tag"]

for index, data in enumerate(train_df["label"].unique()):
    # label 컬럼에 저장된 data 를 index로 변환
    train_df["label"].replace(data, index, inplace=True) 

In [ ]:
train_df = sklearn.utils.shuffle(train_df)

In [ ]:
# 비디오를 구성하는 이미지의 가운데 부분을 리턴
# frame : 비디오를 구성하는 이미지
def crop_center_square(frame):
    # frame.shape[0:2] : 이미지의 세로, 가로 리턴
    y, x = frame.shape[0:2]
    # 이미지의 세로 가로 중에서 작은 값을 리턴
    min_dim = min(y, x)
    #이미지의 왼쪽 모서리 좌표의 가로 시작점
    # 이미지 가로 좌표 (이미지 가로 //2  - min_dim//2) ~ (이미지 가로 //2  + min_dim//2) 
    start_x = (x // 2) - (min_dim // 2)
    # 이미지의 왼쪽 모서리 세로 좌표 시작점
    # 이미지 세로 좌표 (이미지 세로 //2  - min_dim//2) ~ (이미지 세로 //2  + min_dim//2) 
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [ ]:
# 비디오 파일을 읽어서 각 프레임을 이미지로 변환해서 리턴
# path : 비디오 파일 경로
# max_frames : 이미지로 변환할 프레임수
# resize=(IMG_SIZE, IMG_SIZE) : 이미지의 가로 세로
def load_video(path, max_frames=20, resize=(IMG_SIZE, IMG_SIZE)):
    # 비디오 파일을 읽어서 이미지로 변환 할 객페
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            # cap.read() : 비디오를 읽어서 리턴
            # ret : 비디오 읽기가 성공했으면 True, 더이상 읽을 비디오 프레임이 없으면 False 가 리턴
            # frame : 비디오 프레임 이미지를 리턴
            ret, frame = cap.read()
            # ret 가 False면 반복 종료
            if not ret:
                break
            # 비디오 이미지를 가운데 리턴
            frame = crop_center_square(frame)
            #비디오 이미지의 가로 세로를 resize=(224,224) 로 변환
            frame = cv2.resize(frame, resize)
            # frame 은  [줄, 칸, B G R ] 로 구성되 있음 
            # frame [ : (모든줄), : (모든칸), R (인덱스2) G (인덱스1) B (인덱스0 )] 리턴
            frame = frame[:, :, [2, 1, 0]]
            # frame을 frames에 추가
            frames.append(frame)
            # frames에 저장된 데이터수가 max_frames와 같으면 종료
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 이미지 변환 종료
        cap.release()
    # frames를 numpy 배열로 변환 해서 리턴
    return np.array(frames)

In [ ]:
# 모든 비디오 파일의 이미지와 종류를 리턴
# video_name : 비디오 파일명
# label: 비디오 파일 종류
# root_dir : 비디오 파일 경로
def prepare_all_videos(video_name , label, root_dir):
    # 비디오 파일 이름의 개수
    num_samples = len(video_name)
    # video_name.values.tolist() : 비디오 파일 이름을 리스트로 변환 해서 리턴
    video_paths = video_name.values.tolist()
    # 비디오 종류를 리턴
    labels = label.values
    # 비디오 종류를 2차원 배열로 변환
    labels = labels.reshape(-1,1)
    # 0으로 초기화된 [비디오 파일개수 * 20 , 224 , 224, 3] 배열 생성
    x = np.zeros(shape=(num_samples*MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3 ), dtype="float32" )
    # 0으로 초기화된 [비디오파일개수 * 20] 인 배열 생성
    y = np.zeros(shape=(num_samples*MAX_SEQ_LENGTH), dtype="float32")
    # 인덱스 초기화
    index = 0
    # 비디오 파일의 개수 만큼 반복
    for idx, path in enumerate(video_paths):
        # 비디오 파일을 읽어서 각 프레임의 이미지를 리턴
        frames = load_video(root_dir+ path)
        # 프레임의 개수 만큼 반복
        for i in range(len(frames)):
            # frame의 i번째 이미지를 x에 추가
            x[index] = np.array(frames[i], dtype="float32")
            # 비디오의 종류가 저장된 labels의 idx 번째를 y에 추가
            y[index]= np.array(labels[idx], dtype="float32")
            # index 1 증가
            index += 1 
    
    # 전체 비디오 프레임을 이미지로 변환한 x 
    # to_categorical(y) : 이미지의 종류 y 를 onehot 인코딩 
    return (x,to_categorical(y))

In [ ]:
X_train, y_train = prepare_all_videos(train_df["video_name"], train_df["label"] , data_path + "train/")

#증강하지 않은 모델

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [ ]:
vggnet = VGG16( include_top=False,input_shape=(224, 224, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
vggnet.trainable=False

In [ ]:
model = Sequential()
model.add(vggnet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(3, activation="softmax"))

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
                  loss="categorical_crossentropy", 
                   optimizer=Adam(lr=1e-4),
                   metrics=["acc"]
              )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)
save = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_normal.h5", monitor = 'loss', save_best_only=True)
callback = [es, save]

In [ ]:
hist = model.fit(
	#train_sequence가 리턴하는 증강된 이미지를 학습  
	X_train/255, y_train, batch_size = 32,
    # 5번 반복해서 전체 이미지 학습 
	epochs = 100, callbacks = callback)

Epoch 1/100
223/223 [==============================] - 27s 82ms/step - loss: 0.1399 - acc: 0.9552
Epoch 2/100
223/223 [==============================] - 19s 86ms/step - loss: 0.0099 - acc: 0.9990
Epoch 3/100
223/223 [==============================] - 20s 89ms/step - loss: 0.0051 - acc: 0.9997
Epoch 4/100
223/223 [==============================] - 20s 88ms/step - loss: 0.0031 - acc: 0.9999
Epoch 5/100
223/223 [==============================] - 19s 87ms/step - loss: 0.0025 - acc: 1.0000
Epoch 6/100
223/223 [==============================] - 19s 86ms/step - loss: 0.0017 - acc: 1.0000
Epoch 7/100
223/223 [==============================] - 20s 88ms/step - loss: 0.0013 - acc: 1.0000
Epoch 8/100
223/223 [==============================] - 19s 85ms/step - loss: 9.6502e-04 - acc: 1.0000
Epoch 9/100
223/223 [==============================] - 19s 85ms/step - loss: 7.9224e-04 - acc: 1.0000
Epoch 10/100
223/223 [==============================] - 18s 79ms/step - loss: 0.0010 - acc: 1.0000
Epoch 11/10

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_normal.h5")

In [ ]:
from keras.callbacks import EarlyStopping
# 테스트 비디오 파일명과 종류 리턴
test_df = pd.read_csv(data_path+"test.csv")

In [ ]:
test_df["label"] = test_df["tag"]

In [ ]:
text_label = test_df["label"].unique()
# enumerate(test_df["label"].unique()) : label 컬럼에 저장된 데이터를 중복을 제거한 값에 인덱스 추가
for index, data in enumerate(test_df["label"].unique()):
    # label 컬럼에 저장된 data 를 index로 변환
    test_df["label"].replace(data, index, inplace=True) 

In [ ]:
test_df=sklearn.utils.shuffle(test_df)
# sklearn 배열 혼합

In [ ]:
X_test, y_test = prepare_all_videos(test_df["video_name"], test_df["label"] , data_path + "test/")

In [ ]:
model.evaluate(
    X_test/255, y_test, batch_size = 32
)
# .evaluate 함수를 통해 모델 정답률, loss값 도출

86/86 [==============================] - 7s 84ms/step - loss: 0.3487 - acc: 0.8788


[0.3487128019332886, 0.8788321018218994]

accuracy 0.8788, loss 0.3487

# 데이터 증강

In [ ]:
!pip install six numpy scipy Pillow matplotlib scikit-image opencv-python imageio Shapely

In [ ]:
!pip install imgaug

In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.

seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

In [ ]:
import math

In [ ]:
# 이미지를 증강시켜서 배치 사이즈 만큼씩 리턴하는 클래스 구현

class  ImageSequence(tf.keras.utils.Sequence):
    # 객체 생성시 실행되는 함수
    # 매개변수
    # image_arr : 이미지가 저장된 배열
    # label_arr : 이미지의 종류가 저장된 배열
    # batch_size : 배치 사이즈 (한번에 리턴할 파일 개수)
    # seq : 이미지 증강 객체

    def __init__(self, image_arr ,label_arr, batch_size, seq):
        # 매개변수들을 속성에 저장
        self.image_arr  = image_arr 
        self.label_arr = label_arr
        self.batch_size = batch_size
        self.seq = seq
    
    # batch_size씩 이미지를 리턴했을때 전체 이미지를 리턴하려명 몆번 반복해야 하는지 리턴
    def __len__(self):
        
        # math.ceil : 소숫점 1자리 올림 예) 6.1 -> 7   6 -> 6   6.0 -> 6

        # image_arr  (이미지가 저장된 배열) / batch_size (한번에 리턴할 이미지 개수) 의 올림을 리턴     
        return math.ceil(len(self.image_arr ) / self.batch_size)

    # 학습시 batch_size 씩 이미지를 리턴하는 함수로 텐서플로우에서 학습시 model 객체에서 자동으로 호출하는 함수
    # 매개변수 idx : 몆번째 batch 인지가 저장되는 매개변수 0부터 시작
    def __getitem__(self, idx):
        # idx * self.batch_size 부터 (idx + 1) * self.batch_size 까지 이미지 리턴
        batch_image_arr = self.image_arr[idx * self.batch_size:(idx + 1) * self.batch_size]
        # 증강시키기 위해서 batch_image_arr을 정수로 변환
        img = np.array( batch_image_arr,  dtype=np.uint8)
        # img를 증강시켜서 generate_img에 저장
        generate_img = seq(images=img) 
        # idx * self.batch_size 부터 (idx + 1) * self.batch_size 까지 이미지 종류 리턴
        batch_label_arr = self.label_arr[idx * self.batch_size:(idx + 1) * self.batch_size]

        return generate_img/255.0, batch_label_arr
        

In [ ]:
train_sequence = ImageSequence(X_train, y_train, 32, seq)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [ ]:
vggnet = VGG16( include_top=False,input_shape=(224, 224, 3))

In [ ]:
vggnet.trainable=False

In [ ]:
model = Sequential()
model.add(vggnet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(3, activation="softmax"))

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
                  loss="categorical_crossentropy", 
                   optimizer=Adam(lr=1e-4),
                   metrics=["acc"]
              )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)
save = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_aug.h5", monitor = 'loss', save_best_only=True)
callback = [es, save]

In [ ]:
hist = model.fit(
	#train_sequence가 리턴하는 증강된 이미지를 학습  
	train_sequence,
    # 5번 반복해서 전체 이미지 학습 
	epochs = 100, callbacks = callback)

/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:191: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  segments = segmentation.slic(image, n_segments=n_segments_samples[i], compactness=10)


Epoch 1/100
223/223 [==============================] - 111s 496ms/step - loss: 0.9201 - acc: 0.6233
Epoch 2/100
223/223 [==============================] - 118s 529ms/step - loss: 0.5343 - acc: 0.7868
Epoch 3/100
223/223 [==============================] - 115s 514ms/step - loss: 0.4223 - acc: 0.8360
Epoch 4/100
223/223 [==============================] - 110s 495ms/step - loss: 0.3788 - acc: 0.8569
Epoch 5/100
223/223 [==============================] - 113s 508ms/step - loss: 0.3272 - acc: 0.8771
Epoch 6/100
223/223 [==============================] - 115s 516ms/step - loss: 0.3191 - acc: 0.8782
Epoch 7/100
223/223 [==============================] - 116s 522ms/step - loss: 0.2861 - acc: 0.8909
Epoch 8/100
223/223 [==============================] - 113s 504ms/step - loss: 0.2866 - acc: 0.8969
Epoch 9/100
223/223 [==============================] - 112s 504ms/step - loss: 0.2612 - acc: 0.9114
Epoch 10/100
223/223 [==============================] - 112s 502ms/step - loss: 0.2844 - acc: 0.8966

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_aug.h5")

In [ ]:
model.evaluate(
    X_test/255, y_test, batch_size = 32
)

86/86 [==============================] - 7s 78ms/step - loss: 0.2271 - acc: 0.9186


[0.22714002430438995, 0.9186131358146667]

accuracy 0.9186, loss 0.2271

#데이터 증강 + cutmix

In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.

seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

In [ ]:
import math

In [ ]:
# 이미지를 증강시켜서 배치 사이즈 만큼씩 리턴하는 클래스 구현

class  ImageSequence(tf.keras.utils.Sequence):
    # 객체 생성시 실행되는 함수
    # 매개변수
    # image_arr : 이미지가 저장된 배열
    # label_arr : 이미지의 종류가 저장된 배열
    # batch_size : 배치 사이즈 (한번에 리턴할 파일 개수)
    # seq : 이미지 증강 객체

    def __init__(self, image_arr ,label_arr, batch_size, seq):
        # 매개변수들을 속성에 저장
        self.image_arr  = image_arr 
        self.label_arr = label_arr
        self.batch_size = batch_size
        self.seq = seq
    
    # batch_size씩 이미지를 리턴했을때 전체 이미지를 리턴하려명 몆번 반복해야 하는지 리턴
    def __len__(self):
        
        # math.ceil : 소숫점 1자리 올림 예) 6.1 -> 7   6 -> 6   6.0 -> 6

        # image_arr  (이미지가 저장된 배열) / batch_size (한번에 리턴할 이미지 개수) 의 올림을 리턴     
        return math.ceil(len(self.image_arr ) / self.batch_size)

    # 학습시 batch_size 씩 이미지를 리턴하는 함수로 텐서플로우에서 학습시 model 객체에서 자동으로 호출하는 함수
    # 매개변수 idx : 몆번째 batch 인지가 저장되는 매개변수 0부터 시작
    def __getitem__(self, idx):
        # idx * self.batch_size 부터 (idx + 1) * self.batch_size 까지 이미지 리턴
        images = self.image_arr[idx * self.batch_size:(idx + 1) * self.batch_size]
        # 증강시키기 위해서 batch_image_arr을 정수로 변환
        img = np.array( images,  dtype=np.uint8)
        # img를 증강시켜서 generate_img에 저장
        generate_img = seq(images=img) 
        # idx * self.batch_size 부터 (idx + 1) * self.batch_size 까지 이미지 종류 리턴
        labels = self.label_arr[idx * self.batch_size:(idx + 1) * self.batch_size]

        
        generate_img = generate_img/255
        #====================
       # print("자를꺼여")
        #print("="*100)
        imgs = []; labs = []
        for i in range(len(generate_img)):   #i = 자를 사진 index
          #print("i = ", i)
          APPLY = tf.cast(tf.random.uniform(()) >= 0.5, tf.int32)  #0-1 랜덤 숫자 0.5보다 크면 1 아니면 0
          idx = tf.random.uniform((), 0, len(generate_img), tf.int32)  #채워줄 사진 index,  0-배치 중 하나

          W = 224
          H = 224
          lam = tf.random.uniform(())  #사진 비율
          cut_ratio = tf.math.sqrt(1.-lam)
          cut_w = tf.cast(W * cut_ratio, tf.int32) * APPLY   #0곱해지면 적용 x
          cut_h = tf.cast(H * cut_ratio, tf.int32) * APPLY

          cx = tf.random.uniform((), int(W/8), int(7/8*W), tf.int32)   #자르는 이미지의 중심좌표(중심이 너무 외곽이 안잡히도록)
          cy = tf.random.uniform((), int(H/8), int(7/8*H), tf.int32)

          xmin = tf.clip_by_value(cx - cut_w//2, 0, W)  #0과 끝을 벗어나지 않도록
          ymin = tf.clip_by_value(cy - cut_h//2, 0, H)
          xmax = tf.clip_by_value(cx + cut_w//2, 0, W)
          ymax = tf.clip_by_value(cy + cut_h//2, 0, H)

          #print("출력1")
          mid_left = generate_img[i, ymin:ymax, :xmin, :]
         # print("idx = ", idx)
          mid_mid = generate_img[idx, ymin:ymax, xmin:xmax, :] 
          #print("mid_mid = ", mid_mid)         
          mid_right = generate_img[i, ymin:ymax, xmax:, :]
          #print("mid_right = ", mid_right)   

          middle = tf.concat([mid_left, mid_mid, mid_right], axis=1)
          top = generate_img[i, :ymin, :, :]
          bottom = generate_img[i, ymax:, :, :]
          new_img = tf.concat([top, middle, bottom], axis=0)
          imgs.append(new_img)

          cut_w_mod = xmax - xmin   #자르는 이미지가 기존 이미지의 밖으로 나가는 경우 처리하기위해
          cut_h_mod = ymax - ymin
          alpha = tf.cast((cut_w_mod*cut_h_mod)/(W*H), tf.float32) #실제 잘려진 면적의 비율
          label1 = labels[i]
          label2 = labels[idx]
          new_label = ((1-alpha)*label1 + alpha*label2)
          labs.append(new_label)
         # print("new_label 추가!!!")

        #print("np.array(imgs) = ", np.array(imgs).shape )
        
        # new_imgs = tf.reshape(np.stack(imgs), [-1, 224, 224, 3])
        # new_labs = tf.reshape(np.stack(labs), [-1, 3])

        return np.array(imgs, dtype="float32"), np.array(labs,  dtype="float32")

In [ ]:
train_sequence = ImageSequence(X_train, y_train, 32, seq)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [ ]:
vggnet = VGG16( include_top=False,input_shape=(224, 224, 3))

In [ ]:
vggnet.trainable=False

In [ ]:
model = Sequential()
model.add(vggnet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(3, activation="softmax"))

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
                  loss="categorical_crossentropy", 
                   optimizer=Adam(lr=1e-4),
                   metrics=["acc"]
              )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)
save = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_cutmix.h5", monitor = 'loss', save_best_only=True)
callback = [es, save]

In [ ]:
hist = model.fit(
	#train_sequence가 리턴하는 증강된 이미지를 학습  
	train_sequence,
    # 5번 반복해서 전체 이미지 학습 
	epochs = 100, callbacks = callback)

/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:191: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  segments = segmentation.slic(image, n_segments=n_segments_samples[i], compactness=10)


Epoch 1/100
223/223 [==============================] - 150s 671ms/step - loss: 1.1284 - acc: 0.5261
Epoch 2/100
223/223 [==============================] - 145s 650ms/step - loss: 0.6894 - acc: 0.6989
Epoch 3/100
223/223 [==============================] - 150s 669ms/step - loss: 0.5763 - acc: 0.7626
Epoch 4/100
223/223 [==============================] - 148s 664ms/step - loss: 0.5099 - acc: 0.7940
Epoch 5/100
223/223 [==============================] - 147s 660ms/step - loss: 0.4778 - acc: 0.8024
Epoch 6/100
223/223 [==============================] - 151s 679ms/step - loss: 0.4435 - acc: 0.8218
Epoch 7/100
223/223 [==============================] - 149s 670ms/step - loss: 0.4221 - acc: 0.8226
Epoch 8/100
223/223 [==============================] - 150s 670ms/step - loss: 0.4079 - acc: 0.8331
Epoch 9/100
223/223 [==============================] - 148s 661ms/step - loss: 0.3998 - acc: 0.8364
Epoch 10/100
223/223 [==============================] - 145s 648ms/step - loss: 0.3793 - acc: 0.8462

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/ssac/deep/best_model_cutmix.h5")

In [ ]:
 model.evaluate(
    X_test/255, y_test, batch_size = 32
)

86/86 [==============================] - 7s 84ms/step - loss: 0.2062 - acc: 0.9201


[0.2062472552061081, 0.9200729727745056]

accuracy 0.9201  loss 0.2062